In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import os

In [2]:
folder = "./Visibility nc/"

In [3]:
paths = os.listdir(folder)
vis = [xr.open_dataset(folder + path) for path in paths]

c:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(
c:\ProgramData\anaconda3\envs\tf-gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
folder = "./MAIAC AOD gap filled nc/"
paths = os.listdir(folder)
aod = [xr.open_dataset(folder + path) for path in paths]

In [5]:
folder = "./Era5 Land Assimilated/"
paths = sorted(os.listdir(folder))
era5 = [xr.open_dataset(folder + path) for path in paths]

In [6]:
folder = "./Roads nc/"
paths = sorted(os.listdir(folder))
roads = [xr.open_dataset(folder + path) for path in paths]

In [7]:
folder = "./Vegetation nc/"
paths = sorted(os.listdir(folder))
veg = [xr.open_dataset(folder + path).drop('EVI') for path in paths]

In [8]:
for i in range(len(aod)):
    aod[i]['latitude'] = np.round(aod[i]['latitude'].values,2)
    aod[i]['longitude'] = np.round(aod[i]['longitude'].values,2)

In [9]:
for i in range(len(vis)):
    vis[i]['latitude'] = np.round(vis[i]['latitude'].values,2)
    vis[i]['longitude'] = np.round(vis[i]['longitude'].values,2)

In [10]:
for i in range(len(era5)):
    era5[i]['latitude'] = np.round(era5[i]['latitude'].values,2)
    era5[i]['longitude'] = np.round(era5[i]['longitude'].values,2)

In [11]:
for i in range(len(roads)):
    roads[i]['latitude'] = np.round(roads[i]['latitude'].values,2)
    roads[i]['longitude'] = np.round(roads[i]['longitude'].values,2)

In [12]:
for i in range(len(veg)):
    veg[i]['latitude'] = np.round(veg[i]['latitude'].values,2)
    veg[i]['longitude'] = np.round(veg[i]['longitude'].values,2)

In [13]:
nc_data = xr.merge([xr.concat(vis, dim='time'),
                    xr.concat(aod, dim='time'),
                    xr.concat(era5, dim='time'),
                    xr.concat(roads, dim='time'),
                    xr.concat(veg, dim="time")])

In [14]:
nc_data

<xarray.Dataset>
Dimensions:                (time: 1523, latitude: 121, longitude: 191)
Coordinates:
  * time                   (time) datetime64[ns] 2020-12-01 ... 2025-01-31
  * latitude               (latitude) float64 31.0 31.01 31.02 ... 32.19 32.2
  * longitude              (longitude) float64 35.5 35.51 35.52 ... 37.39 37.4
Data variables: (12/15)
    VIS                    (time, latitude, longitude) float64 nan nan ... nan
    filled_AOD             (time, latitude, longitude) float32 nan nan ... nan
    d2m                    (time, longitude, latitude) float32 283.2 ... 272.4
    t2m                    (time, longitude, latitude) float32 291.3 ... 283.3
    u10                    (time, longitude, latitude) float32 0.06851 ... 3.489
    v10                    (time, longitude, latitude) float32 -0.163 ... 0.8076
    ...                     ...
    Primary angle_deg      (time, latitude, longitude) float32 nan nan ... nan
    Secondary distance_km  (time, latitude, longitude) float32 nan nan ... nan
    Secondary angle_deg    (time, latitude, longitude) float32 nan nan ... nan
    Trunk distance_km      (time, latitude, longitude) float32 nan nan ... nan
    Trunk angle_deg        (time, latitude, longitude) float32 nan nan ... nan
    NDVI                   (time, latitude, longitude) float64 nan nan ... nan

In [15]:
for var in list(nc_data.data_vars):
    nc_data[var].values = np.round(nc_data[var].values, 16).astype(np.float16)
#nc_data['latitude'] = np.array(nc_data['latitude'].values, dtype=np.float16)
#nc_data['longitude'] = np.array(nc_data['longitude'].values, dtype=np.float16)

In [16]:
nc_data

<xarray.Dataset>
Dimensions:                (time: 1523, latitude: 121, longitude: 191)
Coordinates:
  * time                   (time) datetime64[ns] 2020-12-01 ... 2025-01-31
  * latitude               (latitude) float64 31.0 31.01 31.02 ... 32.19 32.2
  * longitude              (longitude) float64 35.5 35.51 35.52 ... 37.39 37.4
Data variables: (12/15)
    VIS                    (time, latitude, longitude) float16 nan nan ... nan
    filled_AOD             (time, latitude, longitude) float16 nan nan ... nan
    d2m                    (time, longitude, latitude) float16 283.2 ... 272.2
    t2m                    (time, longitude, latitude) float16 291.2 ... 283.2
    u10                    (time, longitude, latitude) float16 0.06854 ... 3.488
    v10                    (time, longitude, latitude) float16 -0.163 ... 0.8076
    ...                     ...
    Primary angle_deg      (time, latitude, longitude) float16 nan nan ... nan
    Secondary distance_km  (time, latitude, longitude) float16 nan nan ... nan
    Secondary angle_deg    (time, latitude, longitude) float16 nan nan ... nan
    Trunk distance_km      (time, latitude, longitude) float16 nan nan ... nan
    Trunk angle_deg        (time, latitude, longitude) float16 nan nan ... nan
    NDVI                   (time, latitude, longitude) float16 nan nan ... nan

In [17]:
# Get coordinates (assuming lat and lon are the variable names)
latitudes = nc_data['latitude'].values
longitudes = nc_data['longitude'].values

In [18]:
# Define a function to find the nearest grid point
def find_nearest(lat, lon, latitudes, longitudes):
    lat_idx = (np.abs(latitudes - lat)).argmin()
    lon_idx = (np.abs(longitudes - lon)).argmin()
    return lat_idx, lon_idx

In [19]:
# Iterate through Excel files
folder = './AQ2/'
excel_files = sorted(os.listdir(folder))
aq_s = [pd.read_excel(folder + file) for file in excel_files]

In [20]:
met_dat = pd.read_excel('MetaData1.xlsx')

In [21]:
met_dat.sort_values('Symbol', inplace=True)
met_dat.reset_index(inplace=True, drop=True)

In [22]:
met_dat['lat_c'] = np.nan
met_dat['lon_c'] = np.nan
for i in range(len(met_dat)):
    idxs = find_nearest(met_dat['Latitude'][i], met_dat['Longitude'][i], latitudes, longitudes)
    met_dat['lat_c'][i] = latitudes[idxs[0]]
    met_dat['lon_c'][i] = longitudes[idxs[1]]

C:\Users\user\AppData\Local\Temp\ipykernel_9832\1061388935.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  met_dat['lat_c'][i] = latitudes[idxs[0]]
C:\Users\user\AppData\Local\Temp\ipykernel_9832\1061388935.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  met_dat['lon_c'][i] = longitudes[idxs[1]]


In [23]:
for i in range(len(aq_s)):
    aq_s[i]['latitude'] = met_dat['lat_c'][i]
    aq_s[i]['longitude'] = met_dat['lon_c'][i]
    aq_s[i]['time'] = aq_s[i]['date']

In [24]:
aq_mon = pd.concat(aq_s).set_index(['time', 'latitude', 'longitude']).drop('date', axis=1)

In [25]:
nc_df = nc_data.to_dataframe()

In [26]:
nc_df.reset_index(inplace=True)

In [27]:
nc_df.sort_values(['time', 'latitude', 'longitude'], ascending=[True, False, True], inplace=True)
nc_df.reset_index(inplace=True, drop=True)

In [28]:
len(nc_df['longitude'].unique())

191

In [29]:
def time_lag_features(in_df, col_name, lag_steps):
    lag = len(in_df[in_df['time'] == in_df['time'].unique()[0]])
    for i in range(lag_steps):
        in_df[col_name + '_' + f'lag{i+1}'] = in_df[col_name].shift(lag * (i+1))
        in_df[col_name + '_' + f'lag-{i+1}'] = in_df[col_name].shift(lag * -(i+1))
    return in_df

In [30]:
del aod, era5, vis, nc_data

In [31]:
nc_df = time_lag_features(nc_df, 'filled_AOD', 7)
nc_df = time_lag_features(nc_df, 'u10', 2)
nc_df = time_lag_features(nc_df, 'v10', 2)

In [32]:
def time_lag_features(in_df, col_name, shift_list):
    len_lons = len(in_df['longitude'].unique())
    for i in shift_list:
        in_df[col_name + '_' + f'u{i}'] = in_df[col_name].shift(len_lons * (i))
        in_df[col_name + '_' + f'lw{i}'] = in_df[col_name].shift(-len_lons * (i))
        in_df[col_name + '_' + f'le{i}'] = in_df[col_name].shift((i))
        in_df[col_name + '_' + f'r{i}'] = in_df[col_name].shift(-(i))
        in_df[col_name + '_' + f'u-le{i}'] = in_df[col_name].shift(len_lons * (i) + (i))
        in_df[col_name + '_' + f'lw-r{i}'] = in_df[col_name].shift(-len_lons * (i) - (i))
        in_df[col_name + '_' + f'u-r{i}'] = in_df[col_name].shift(len_lons * (i) - (i))
        in_df[col_name + '_' + f'lw-le{i}'] = in_df[col_name].shift(-len_lons * (i) + (i))
    return in_df

In [33]:
nc_df.columns

Index(['time', 'latitude', 'longitude', 'VIS', 'filled_AOD', 'd2m', 't2m',
       'u10', 'v10', 'Motorway distance_km', 'Motorway angle_deg',
       'Primary distance_km', 'Primary angle_deg', 'Secondary distance_km',
       'Secondary angle_deg', 'Trunk distance_km', 'Trunk angle_deg', 'NDVI',
       'filled_AOD_lag1', 'filled_AOD_lag-1', 'filled_AOD_lag2',
       'filled_AOD_lag-2', 'filled_AOD_lag3', 'filled_AOD_lag-3',
       'filled_AOD_lag4', 'filled_AOD_lag-4', 'filled_AOD_lag5',
       'filled_AOD_lag-5', 'filled_AOD_lag6', 'filled_AOD_lag-6',
       'filled_AOD_lag7', 'filled_AOD_lag-7', 'u10_lag1', 'u10_lag-1',
       'u10_lag2', 'u10_lag-2', 'v10_lag1', 'v10_lag-1', 'v10_lag2',
       'v10_lag-2'],
      dtype='object')

In [34]:
s_lis_aod = [1, 2, 3, 4, 5, 6, 7, 8]
cols_aod = ['filled_AOD', 'filled_AOD_lag1', 'filled_AOD_lag-1', 
            'filled_AOD_lag2', 'filled_AOD_lag-2', 'filled_AOD_lag3', 'filled_AOD_lag-3',
            'filled_AOD_lag4', 'filled_AOD_lag-4', 'filled_AOD_lag5',
       'filled_AOD_lag-5']

In [35]:
s_lis_era5 = [10, 20, 30, 40]
cols_era5 = ['u10', 'v10', 'u10_lag1', 'v10_lag1']

In [36]:
for col in cols_aod:
    nc_df = time_lag_features(nc_df, col, s_lis_aod)
for col in cols_era5:
    nc_df = time_lag_features(nc_df, col, s_lis_era5)

nc_df.set_index(['time', 'latitude', 'longitude'], inplace=True)

fin = []
for sta in aq_mon['Abr'].unique():
    fin.append(pd.concat([nc_df,aq_mon[aq_mon['Abr'] == sta]], axis=1, join='inner'))

fin = pd.concat(fin).drop(['PM10', 'NO2', 'SO2', 'CO', 'O3', 'AQI', 'T', 'RH', 'WD', 'WS'], axis=1).dropna()
del nc_df

C:\Users\user\AppData\Local\Temp\ipykernel_9832\144542073.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  in_df[col_name + '_' + f'lw-r{i}'] = in_df[col_name].shift(-len_lons * (i) - (i))
C:\Users\user\AppData\Local\Temp\ipykernel_9832\144542073.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  in_df[col_name + '_' + f'u-r{i}'] = in_df[col_name].shift(len_lons * (i) - (i))
C:\Users\user\AppData\Local\Temp\ipykernel_9832\144542073.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of cal

In [37]:
fin

,,,VIS,filled_AOD,d2m,t2m,u10,v10,Motorway distance_km,Motorway angle_deg,Primary distance_km,Primary angle_deg,...,v10_lag1_u40,v10_lag1_lw40,v10_lag1_le40,v10_lag1_r40,v10_lag1_u-le40,v10_lag1_lw-r40,v10_lag1_u-r40,v10_lag1_lw-le40,Abr,PM2.5
time,latitude,longitude,,,,,,,,,,,,,,,,,,,,,
2021-01-18,31.95,35.93,7668.0,0.161255,277.50,280.50,2.619141,2.013672,18.031250,224.5,112.4375,171.5,...,0.820312,3.960938,2.986328,2.566406,1.087891,3.150391,1.399414,4.527344,GAM,12.85
2021-01-19,31.95,35.93,6520.0,0.125488,276.00,281.00,3.281250,2.628906,18.031250,224.5,112.4375,171.5,...,1.582031,3.146484,1.973633,1.492188,2.376953,2.333984,2.501953,3.542969,GAM,10.50
2021-01-20,31.95,35.93,7192.0,0.230713,276.25,278.00,2.855469,0.727051,18.031250,224.5,112.4375,171.5,...,0.937500,3.310547,2.462891,2.410156,1.754883,2.900391,1.361328,3.675781,GAM,12.06
2021-01-21,31.95,35.93,8608.0,0.086609,274.25,278.50,-0.094177,-0.451904,18.031250,224.5,112.4375,171.5,...,1.194336,1.221680,0.603516,-0.596680,1.752930,0.327148,1.974609,1.072266,GAM,17.50
2021-01-22,31.95,35.93,9008.0,0.140381,269.75,278.25,-3.228516,0.685547,18.031250,224.5,112.4375,171.5,...,-0.054016,-0.566406,-0.929199,-1.016602,0.797363,-0.596680,-0.312500,-1.287109,GAM,9.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-27,31.94,35.94,7000.0,0.299072,270.75,281.25,3.238281,2.853516,10.796875,233.0,113.5000,171.0,...,2.501953,0.641602,0.804199,0.087219,0.200439,0.148926,4.121094,0.778320,YAR,36.20
2024-12-28,31.94,35.94,6672.0,0.186523,273.25,281.75,1.970703,2.916016,10.796875,233.0,113.5000,171.0,...,-0.049255,2.914062,3.062500,1.946289,0.201294,2.054688,0.115356,3.693359,YAR,33.80
2024-12-29,31.94,35.94,7044.0,0.117310,273.75,281.75,1.105469,1.903320,10.796875,233.0,113.5000,171.0,...,1.236328,3.076172,2.873047,2.507812,2.185547,2.687500,1.435547,3.611328,YAR,21.00


In [38]:
'''# Separate numeric and string columns
numeric_cols = fin.select_dtypes(include=['float64']).columns

merged = fin.reset_index().groupby(['time', 'latitude', 'longitude']).agg(
    {col: 'mean' for col in numeric_cols})

merged
'''

"# Separate numeric and string columns\nnumeric_cols = fin.select_dtypes(include=['float64']).columns\n\nmerged = fin.reset_index().groupby(['time', 'latitude', 'longitude']).agg(\n    {col: 'mean' for col in numeric_cols})\n\nmerged\n"

In [39]:
fin.reset_index(inplace=True)

In [40]:
fin['Day_of_Week'] = fin['time'].dt.day_of_week
fin['Day_of_Year'] = fin['time'].dt.day_of_year
fin['Month'] = fin['time'].dt.month
fin['Year'] = fin['time'].dt.year

In [41]:
fin.to_excel('Final_Data 5.xlsx', index=False)